In [8]:
import pandas as pd
import folium
from folium import plugins
from branca.colormap import LinearColormap
from IPython.display import IFrame, display
import json

# Load the dataset
df = pd.read_csv('properties.csv')
df2 = pd.read_csv('PMDen.csv', encoding = "ISO-8859-1")


# Filter rows where 'Address' contains 'Ottawa' (case-insensitive)
filtered_df = df[df['Address'].str.contains('Ottawa',case=False, na=False)]

# Group by 'AreaName' and calculate average price and count of houses
area_stats = filtered_df.groupby('AreaName').agg({
    'Price ($)': 'mean',
    'Address': 'count'  # Assuming 'Address' is the column containing the house info
}).reset_index()
area_stats.rename(columns={'Address': 'HouseCount'}, inplace=True)

#group by ottawa
filtered_df2 = df2[df2['CMANAME'].str.contains('Ottawa', case=False, na=False)]

#filtered_df.drop(filtered_df.columns.difference(['DBUID','DBPOP']), 1, inplace=True)
filtered_df2 = filtered_df2.filter(['prox_idx_emp', 'prox_idx_pharma',
                                'prox_idx_childcare', 'prox_idx_health', 'prox_idx_grocery', 
                                'prox_idx_educpri', 'prox_idx_educsec', 'prox_idx_lib',
                                'prox_idx_parks', 'prox_idx_transit', 'lat', 'lon'])

name_list = filtered_df2.columns.drop(['lat', 'lon'])
filtered_df2[name_list] = filtered_df2[name_list].apply(pd.to_numeric, errors='coerce')

#print(filtered_df.head())


area_stats_info = filtered_df2.groupby(filtered_df2.index // 80).agg({
    'prox_idx_emp': 'mean',
    'prox_idx_pharma': 'mean',
    'prox_idx_childcare': 'mean',
    'prox_idx_health': 'mean',
    'prox_idx_grocery': 'mean',
    'prox_idx_educpri': 'mean',
    'prox_idx_educsec': 'mean',
    'prox_idx_lib': 'mean',
    'prox_idx_parks': 'mean',
    'prox_idx_transit': 'mean',
    'lat': 'mean',
    'lon': 'mean'
}).reset_index()
###########################################

# Create a diverging color map based on average price
colormap = LinearColormap(
    ['#ef8a62', '#f7f7f7', '#67a9cf'],
    vmin=area_stats['Price ($)'].min(),
    vmax=area_stats['Price ($)'].max()
)

colormap_info = LinearColormap(
    ['#fef7ff', '#220029'],
    vmin=area_stats_info['prox_idx_parks'].min(),
    vmax=area_stats_info['prox_idx_parks'].max()
)


# Load the GeoJSON file
with open('geo-area.geojson') as f:
    geo_json_data = json.load(f)

# Create a base map centered around Ottawa with a grey background
m = folium.Map(location=[45.4215, -75.6972], zoom_start=12, tiles='cartodbpositron')

# Function to style the GeoJson layer based on average price
def style_function(feature):
    area_name = feature['properties']['Name']
    value = area_stats.loc[area_stats['AreaName'] == area_name, 'Price ($)']
    if not value.empty:
        color = colormap(value.iloc[0])
    else:
        color = 'white'
    return {
        'fillColor': color,
        'color': 'black',
        'weight': 1,
        'fillOpacity': 0.7
    }
# Function to create pop-ups for the GeoJson layer
def popup_function(feature):
    area_name = feature['properties']['Name']
    value = area_stats.loc[area_stats['AreaName'] == area_name]
    if not value.empty:
        avg_price = value['Price ($)'].iloc[0]
        house_count = value['HouseCount'].iloc[0]
        popup_content = (
            f"<div style='font-size:14px;'>"
            f"Area: {area_name}<br>"
            f"Average Price: ${avg_price:,.2f}<br>"
            f"Number of Houses Sold: {house_count}"
            f"</div>"
        )
    else:
        popup_content = (
            f"<div style='font-size:14px;'>"
            f"Area: {area_name}<br>No data available"
            f"</div>"
        )
    return popup_content

# Adding the GeoJson layer with dynamic styling and pop-ups
for feature in geo_json_data['features']:
    popup = popup_function(feature)
    folium.GeoJson(
        feature,
        style_function=style_function,
        #tooltip=folium.GeoJsonTooltip(fields=['Name', 'Name_FR']),
        popup=folium.Popup(popup, max_width=300)
    ).add_to(m)

def colour_function(row):
    if not pd.isna(row):
        return colormap_info(row)
    else:
        return 'grey'
 
# Add markers to the map
for _, row in area_stats_info.iterrows():
    folium.CircleMarker(
        location=[row['lat'], row['lon']],
        color=colour_function(row['prox_idx_parks']),
        fill_opacity=1,
        fill=True,
        popup=(
            f"proximity to parks:  {row['prox_idx_parks']:,.2f}<br>"
        )
    ).add_to(m)

# Add color legend
colormap_info.add_to(m)

# Add color legend
colormap.add_to(m)

# Save the map to an HTML file
map_file = './content/ottawa_real_state_price_map.html'
m.save(map_file)

# Display the map in Colab
display(IFrame(src=map_file, width=400, height=600))